In [1]:
import os
os.environ["SSL_CERT_FILE"] = "Fortinet_CA_SSL(15).cer"


In [2]:
import requests
import json
from bs4 import BeautifulSoup


In [3]:

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    # Step 2: Grab the lead/description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    lead_text = BeautifulSoup(lead_html, "html.parser").get_text(" ", strip=True)
    result["description"] = lead_text

    # Step 3: Iterate sections
    for sec in res["parse"]["sections"]:
        title = sec["line"].strip().lower()
        if title in wanted:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": sec["index"]
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            text = BeautifulSoup(html, "html.parser").get_text(" ", strip=True)
            result[sec["line"]] = text

    return result

# Example: London
data = fetch_wikivoyage_sections("London", "cairo")

# Save to JSON file
with open("london.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Data saved to london.json")


✅ Data saved to london.json


In [6]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    # Helper: clean HTML → paragraphs
    def html_to_paragraphs(html: str):
        soup = BeautifulSoup(html, "html.parser")
        paragraphs = []
        for p in soup.find_all("p"):
            text = p.get_text(" ", strip=True)
            if text:  # skip empty
                paragraphs.append(text)
        return paragraphs

    # Step 2: Grab the lead/description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    result["description"] = html_to_paragraphs(lead_html)

    # Step 3: Iterate sections
    for sec in res["parse"]["sections"]:
        title = sec["line"].strip()
        if title.lower() in wanted:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": sec["index"]
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            result[title] = html_to_paragraphs(html)

    return result

# Example: London
data = fetch_wikivoyage_sections("London")

# Save to JSON file
with open("london.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Data saved to london.json")


✅ Data saved to london.json


In [7]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections (top + sub)
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()
    sections_meta = res["parse"]["sections"]

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    # Helper: clean HTML → paragraphs
    def html_to_paragraphs(html: str):
        soup = BeautifulSoup(html, "html.parser")
        paragraphs = []
        for p in soup.find_all("p"):
            text = p.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)
        return paragraphs

    # Step 2: Grab description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    result["description"] = html_to_paragraphs(lead_html)

    # Step 3: Iterate through sections to build nested structure
    for sec in sections_meta:
        title = sec["line"].strip()
        title_lower = title.lower()
        number = sec["number"]  # like "9", "9.1", etc.
        index = sec["index"]

        # Top-level section
        if title_lower in wanted and "." not in number:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": index
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            result[title] = {"content": html_to_paragraphs(html), "subsections": {}}

        # Subsection (belongs to the last top-level)
        elif "." in number:
            parent_num = number.split(".")[0]  # e.g. "9.1" → "9"
            parent_section = next((s for s in sections_meta if s["number"] == parent_num), None)
            if parent_section and parent_section["line"].strip().lower() in wanted:
                parent_title = parent_section["line"].strip()
                sec_params = {
                    "action": "parse",
                    "page": page,
                    "format": "json",
                    "prop": "text",
                    "section": index
                }
                sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
                html = sec_data["parse"]["text"]["*"]
                if parent_title in result:
                    result[parent_title]["subsections"][title] = html_to_paragraphs(html)

    return result

# Example: London
data = fetch_wikivoyage_sections("London")

# Save to JSON file
with open("london_subsection.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Data with subsections saved to london_subsection.json")


✅ Data with subsections saved to london_subsection.json


In [12]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections (top + sub)
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()
    sections_meta = res["parse"]["sections"]

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    # Helper: clean HTML → paragraphs

    def html_to_paragraphs(html: str):
        soup = BeautifulSoup(html, "html.parser")
        paragraphs = []
    
        # Get paragraphs
        for p in soup.find_all("p"):
            text = p.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)
    
        # Get list items
        for li in soup.find_all("li"):
            text = li.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)
    
        # Get table cells
        for td in soup.find_all("td"):
            text = td.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)
    
        return paragraphs

    # Step 2: Grab description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    result["description"] = html_to_paragraphs(lead_html)

    # Step 3: Iterate through sections to build nested structure
    for sec in sections_meta:
        title = sec["line"].strip()
        title_lower = title.lower()
        number = sec["number"]  # like "9", "9.1", etc.
        index = sec["index"]

        # Top-level section
        if title_lower in wanted and "." not in number:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": index
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            result[title] = {"content": html_to_paragraphs(html), "subsections": {}}

        # Subsection (belongs to the last top-level)
        elif "." in number:
            parent_num = number.split(".")[0]  # e.g. "9.1" → "9"
            parent_section = next((s for s in sections_meta if s["number"] == parent_num), None)
            if parent_section and parent_section["line"].strip().lower() in wanted:
                parent_title = parent_section["line"].strip()
                sec_params = {
                    "action": "parse",
                    "page": page,
                    "format": "json",
                    "prop": "text",
                    "section": index
                }
                sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
                html = sec_data["parse"]["text"]["*"]
                if parent_title in result:
                    result[parent_title]["subsections"][title] = html_to_paragraphs(html)

    return result

# Example: London
data = fetch_wikivoyage_sections("London")

# Save to JSON file
with open("london_subsection_test1.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Data with subsections saved to london_subsection_test1.json")


✅ Data with subsections saved to london_subsection_test1.json


In [13]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections (top + sub)
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()
    sections_meta = res["parse"]["sections"]

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    # Helper: clean HTML → paragraphs, list items, tables
    def html_to_paragraphs(html: str):
        soup = BeautifulSoup(html, "html.parser")
        paragraphs = []

        for p in soup.find_all("p"):
            text = p.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        for li in soup.find_all("li"):
            text = li.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        for td in soup.find_all("td"):
            text = td.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        return paragraphs

    # Step 2: Grab description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    result["description"] = html_to_paragraphs(lead_html)

    # Step 3: Iterate through sections to build nested structure
    for sec in sections_meta:
        title = sec["line"].strip()
        title_lower = title.lower()
        number = sec["number"]
        index = sec["index"]

        if title_lower in wanted and "." not in number:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": index
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            result[title] = {"content": html_to_paragraphs(html), "subsections": {}}

        elif "." in number:
            parent_num = number.split(".")[0]
            parent_section = next((s for s in sections_meta if s["number"] == parent_num), None)
            if parent_section and parent_section["line"].strip().lower() in wanted:
                parent_title = parent_section["line"].strip()
                sec_params = {
                    "action": "parse",
                    "page": page,
                    "format": "json",
                    "prop": "text",
                    "section": index
                }
                sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
                html = sec_data["parse"]["text"]["*"]
                if parent_title in result:
                    result[parent_title]["subsections"][title] = html_to_paragraphs(html)

    return result

# === Multiple cities ===
cities = ["London", "Rome", "Seoul", "Cairo"]

for city in cities:
    print(f"📍 Fetching data for {city}...")
    data = fetch_wikivoyage_sections(city)
    filename = f"{city.lower()}_subsections.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved {filename}")


📍 Fetching data for London...
✅ Saved london_subsections.json
📍 Fetching data for Rome...
✅ Saved rome_subsections.json
📍 Fetching data for Seoul...
✅ Saved seoul_subsections.json
📍 Fetching data for Cairo...
✅ Saved cairo_subsections.json


In [14]:
import requests
import json
from bs4 import BeautifulSoup

def fetch_wikivoyage_sections(page: str, user_agent="MyWikiVoyageClient/1.0 (contact: your-email@example.com)"):
    api_url = "https://en.wikivoyage.org/w/api.php"
    headers = {"User-Agent": user_agent}

    # Step 1: Get metadata about all sections
    params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "sections"
    }
    res = requests.get(api_url, params=params, headers=headers).json()
    sections_meta = res["parse"]["sections"]

    wanted = {"districts", "see", "buy", "eat", "drink", "sleep", "stay safe"}
    result = {}

    def html_to_paragraphs(html: str):
        soup = BeautifulSoup(html, "html.parser")
        paragraphs = []

        for p in soup.find_all("p"):
            text = p.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        for li in soup.find_all("li"):
            text = li.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        for td in soup.find_all("td"):
            text = td.get_text(" ", strip=True)
            if text:
                paragraphs.append(text)

        return paragraphs

    # Step 2: Grab description (section 0)
    lead_params = {
        "action": "parse",
        "page": page,
        "format": "json",
        "prop": "text",
        "section": 0
    }
    lead = requests.get(api_url, params=lead_params, headers=headers).json()
    lead_html = lead["parse"]["text"]["*"]
    result["description"] = html_to_paragraphs(lead_html)

    # Step 3: Iterate through sections
    for sec in sections_meta:
        title = sec["line"].strip()
        title_lower = title.lower()
        number = sec["number"]
        index = sec["index"]

        if title_lower in wanted and "." not in number:
            sec_params = {
                "action": "parse",
                "page": page,
                "format": "json",
                "prop": "text",
                "section": index
            }
            sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
            html = sec_data["parse"]["text"]["*"]
            content = html_to_paragraphs(html)
            result[title] = {"content": content, "subsections": {}}

        elif "." in number:
            parent_num = number.split(".")[0]
            parent_section = next((s for s in sections_meta if s["number"] == parent_num), None)
            if parent_section and parent_section["line"].strip().lower() in wanted:
                parent_title = parent_section["line"].strip()
                sec_params = {
                    "action": "parse",
                    "page": page,
                    "format": "json",
                    "prop": "text",
                    "section": index
                }
                sec_data = requests.get(api_url, params=sec_params, headers=headers).json()
                html = sec_data["parse"]["text"]["*"]
                content = html_to_paragraphs(html)
                if parent_title in result and content:
                    result[parent_title]["subsections"][title] = content

    # 🔹 Cleanup: remove empty subsections
    cleaned = {}
    for k, v in result.items():
        if isinstance(v, dict):
            # Drop empty subsections
            subsections = {sk: sv for sk, sv in v.get("subsections", {}).items() if sv}
            if v.get("content") or subsections:
                cleaned[k] = {"content": v.get("content", []), "subsections": subsections}
        elif v:  # e.g. description
            cleaned[k] = v

    return cleaned

# === Multiple cities ===
cities = ["London", "Rome", "Seoul", "Cairo"]

for city in cities:
    print(f"📍 Fetching data for {city}...")
    data = fetch_wikivoyage_sections(city)
    filename = f"{city.lower()}_subsections_1.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved {filename}")


📍 Fetching data for London...
✅ Saved london_subsections_1.json
📍 Fetching data for Rome...
✅ Saved rome_subsections_1.json
📍 Fetching data for Seoul...
✅ Saved seoul_subsections_1.json
📍 Fetching data for Cairo...
✅ Saved cairo_subsections_1.json


In [30]:
import pandas as pd
import csv

In [35]:
import json
import csv
import glob

def json_to_csv(json_file, csv_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []

    city = data.get("metadata", {}).get("city")
    country = data.get("metadata", {}).get("country")
    continent = data.get("metadata", {}).get("continent")

    # Description
    for text in data.get("description", []):
        rows.append([city, country, continent, "description", None, text])

    # Sections + subsections
    for section, sec_data in data.items():
        if section in ["description", "metadata"]:
            continue
        content = sec_data.get("content", [])
        for text in content:
            rows.append([city, country, continent, section, None, text])
        subsections = sec_data.get("subsections", {})
        for sub, sub_content in subsections.items():
            for text in sub_content:
                rows.append([city, country, continent, section, sub, text])

    # Write CSV
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["city", "country", "continent", "section", "subsection", "text"])
        writer.writerows(rows)

# === Convert all generated JSONs to CSV ===
for json_file in glob.glob("*_subsections.json"):
    csv_file = json_file.replace(".json", ".csv")
    json_to_csv(json_file, csv_file)
    print(f"✅ Converted {json_file} → {csv_file}")


✅ Converted cairo_subsections.json → cairo_subsections.csv
✅ Converted london_subsections.json → london_subsections.csv
✅ Converted rome_subsections.json → rome_subsections.csv
✅ Converted seoul_subsections.json → seoul_subsections.csv


In [38]:
import json
import csv
import glob
import re
import os
import requests

USER_AGENT = "MyWikiVoyageClient/1.0 (contact: your-email@example.com)"
HEADERS = {"User-Agent": USER_AGENT}

# 1) small known mapping for quick, accurate results
KNOWN_CITY_MAP = {
    "london": ("United Kingdom", "Europe"),
    "rome": ("Italy", "Europe"),
    "seoul": ("South Korea", "Asia"),
    "cairo": ("Egypt", "Africa"),
    # add more known cities here if you want
}

# 2) fallback continent map for some countries (used if pycountry_convert not installed)
FALLBACK_COUNTRY_TO_CONTINENT = {
    "United Kingdom": "Europe",
    "England": "Europe",
    "Italy": "Europe",
    "South Korea": "Asia",
    "Egypt": "Africa",
}

def convert_json_to_csv(json_file, csv_file, fill_geo=True):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # prefer metadata.city if available, else infer from filename
    meta_city = data.get("metadata", {}).get("city")
    if meta_city:
        city = meta_city
    else:
        # extract from filename e.g. "london_subsections.json" -> "London"
        base = os.path.basename(json_file)
        m = re.match(r'(.+?)_subsections\.json$', base, flags=re.I)
        if m:
            city_raw = m.group(1)
        else:
            city_raw = os.path.splitext(base)[0]
        city = city_raw.replace("_", " ").strip().title()

    # infer geo
    country = None
    continent = None
    if fill_geo:
        country, continent = infer_country_continent(city)
    # If still None and there is metadata, try metadata.country/continent
    if (not country or not continent) and data.get("metadata"):
        country = country or data["metadata"].get("country")
        continent = continent or data["metadata"].get("continent")

    rows = []
    # Description rows
    for text in data.get("description", []):
        rows.append([city, country or "", continent or "", "description", "", text])

    # Sections
    for section, sec_data in data.items():
        if section in ("description", "metadata"):
            continue
        if not isinstance(sec_data, dict):
            continue
        # top-level content
        for txt in sec_data.get("content", []):
            rows.append([city, country or "", continent or "", section, "", txt])
        # subsections
        for sub, sub_list in sec_data.get("subsections", {}).items():
            for txt in sub_list:
                rows.append([city, country or "", continent or "", section, sub, txt])

    # Write per-file CSV
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["city", "country", "continent", "section", "subsection", "text"])
        writer.writerows(rows)

    return rows  # return rows so we can append to combined CSV if desired

# === main: convert all *_subsections.json files and create combined CSV ===
all_rows = []
files = glob.glob("*_subsections.json")
if not files:
    print("No '*_subsections.json' files found in current directory.")
else:
    for jf in files:
        csv_name = jf.replace(".json", ".csv")
        print(f"Converting {jf} -> {csv_name} ...")
        rows = convert_json_to_csv(jf, csv_name, fill_geo=True)
        all_rows.extend(rows)
        print(f"  rows written: {len(rows)}")

    # write combined CSV
    combined_name = "all_cities_combined.csv"
    print(f"Writing combined CSV: {combined_name} (total rows: {len(all_rows)})")
    with open(combined_name, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["city", "country", "continent", "section", "subsection", "text"])
        writer.writerows(all_rows)

    print("✅ Done. Per-file CSVs plus combined CSV created.")


Converting cairo_subsections.json -> cairo_subsections.csv ...
  rows written: 109
Converting london_subsections.json -> london_subsections.csv ...
  rows written: 468
Converting rome_subsections.json -> rome_subsections.csv ...
  rows written: 252
Converting seoul_subsections.json -> seoul_subsections.csv ...
  rows written: 71
Writing combined CSV: all_cities_combined.csv (total rows: 900)
✅ Done. Per-file CSVs plus combined CSV created.


In [40]:
import csv

def count_words_in_csv_rows(file_path):
    """
    Counts the number of words in each row of a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        list: A list of integers, where each integer represents the word count for a corresponding row.
    """
    word_counts = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            row_word_count = 0
            for cell in row:
                # Split the cell content by whitespace to get words
                words = cell.split()
                row_word_count += len(words)
            word_counts.append(row_word_count)
    return word_counts

# Example usage:
csv_file = 'all_cities_combined.csv'  # Replace with the actual path to your CSV file
row_word_counts = count_words_in_csv_rows(csv_file)

for i, count in enumerate(row_word_counts):
    print(f"Row {i+1}: {count} words")

Row 1: 6 words
Row 2: 77 words
Row 3: 106 words
Row 4: 111 words
Row 5: 26 words
Row 6: 74 words
Row 7: 23 words
Row 8: 20 words
Row 9: 51 words
Row 10: 22 words
Row 11: 34 words
Row 12: 76 words
Row 13: 19 words
Row 14: 47 words
Row 15: 14 words
Row 16: 28 words
Row 17: 24 words
Row 18: 40 words
Row 19: 87 words
Row 20: 69 words
Row 21: 21 words
Row 22: 27 words
Row 23: 25 words
Row 24: 27 words
Row 25: 24 words
Row 26: 30 words
Row 27: 16 words
Row 28: 26 words
Row 29: 35 words
Row 30: 40 words
Row 31: 80 words
Row 32: 19 words
Row 33: 82 words
Row 34: 21 words
Row 35: 67 words
Row 36: 55 words
Row 37: 154 words
Row 38: 50 words
Row 39: 86 words
Row 40: 27 words
Row 41: 43 words
Row 42: 127 words
Row 43: 34 words
Row 44: 173 words
Row 45: 63 words
Row 46: 63 words
Row 47: 80 words
Row 48: 38 words
Row 49: 26 words
Row 50: 56 words
Row 51: 179 words
Row 52: 109 words
Row 53: 68 words
Row 54: 143 words
Row 55: 67 words
Row 56: 21 words
Row 57: 5 words
Row 58: 6 words
Row 59: 5 words
Ro